In [ ]:
import sys, os
import numpy as np

sys.path.insert(0, "/Users/Shared/auto/07p/python")
#sys.path.insert(0, "/Users/ielyaascloete/auto/07p/python")
from auto import *


In [ ]:
cl

In [ ]:
import sys
print(sys.version)

In [ ]:
Eq = run(e='PKC_ipr_m1',c='PKC_ipr_m1.eq')

In [ ]:
PO1 = run(Eq('HB1'),c ='PKC_ipr_m1.per',SP=['BP0'])

In [ ]:
#PO2 = run(Eq('HB2'),c ='PKC_ipr_m1.per',DS='-',SP=[''])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def auto_to_csv(branch,name,solution_type):
    """ The goal of this function is to save as a csv file the results of an AUTO continuation 
        and return a panda dataframe of the data"""
    """ Input:
        branch: an AUTO branch solution acquired using run(blablabla)
        name: The name of the output csv file, must be a string
        solution_type: 'EQ' for equilibrium, 'PR' for periodic orbit, 'HB' for Hopf two parameters continuation
            'LPPR' Limit point of periodic orbit
        
        Output:
        panda dataframe of the branch solution"""
    
    
    """Relabelling the branch solution """
    branch = rl(branch)
    """Saving the branch solution"""
    sv(branch,name)
    
    """Acquire the b. file"""
    print("Acquire the b. file")
    name_b = 'b.' + name
    content = None
    with open(name_b, 'r') as f:
        content = f.readlines()
    
    
    print("Read file")
    """Read the file from the beginning of the interesting part (depends on its type)"""
    
    for i in range(len(content)):
    
        if "PT" in content[i]:
            init = i
            break



            
    content_csv = [[el for el in content[init].split(' ') if len(el) > 0 and el != '\n']]
    print(content_csv)
        
    
        
    print("Rename branch")
    """ Rename the branch"""
    content_csv[0][0] = 'branch'
    column_names = content_csv[0]
    
    
    
    print("Split content")
    """Split and read the content"""
    for line in content:
        dummy = line.split(' ')
        dummy = [el for el in dummy if len(el) > 0 and el != '\n']
        if dummy[0] == '0':
            continue

        for el_i, el in enumerate(dummy):
            if el_i < 4:
                dummy[el_i] = int(el)
            else:
                dummy[el_i] = float(el)

        if len(dummy) > 1:
            content_csv.append(dummy)
        
    # transform into panda data frame
        
    df = pd.DataFrame(content_csv, columns=column_names)
    
    # Export to csv
    name_csv = name + '.csv'
    df[1:-1].to_csv(name_csv)

        
    
    
    
    return df[1:-1]

In [ ]:
df_eq = auto_to_csv(Eq,'Eq','EQ')

In [ ]:
df_PR = auto_to_csv(PO1,'PO1','PR')

In [ ]:
import plotly.plotly as py
from plotly.graph_objs import *
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [ ]:
trace_eq = Scatter(x=df_eq['V_plc'],
                  y=df_eq['PAR(5)'],
                  name='Equilibria',
                  mode = 'lines')
trace_per = Scatter(x=df_PR['V_plc'],
                  y=df_PR['PAR(5)'],
                  name='Periodic Orbit',
                   mode='lines')
trace_per_min = Scatter(x=df_PR['V_plc'],
                  y=df_PR['PAR(32)'],
                  name='Periodic Orbit',
                   mode='lines')
data = [trace_eq,trace_per,trace_per_min]
layout = Layout(title="Bifurcation diagram",
               xaxis=dict(title='V_plc'),
               yaxis=dict(title='[Ca]'))
fig = Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
trace_eq = Scatter(x=df_eq['V_plc'],
                  y=df_eq['L2-NORM'],
                  name='Equilibria',
                  mode = 'lines')
trace_per = Scatter(x=df_PR['V_plc'],
                  y=df_PR['L2-NORM'],
                  name='Periodic Orbit',
                   mode='lines')
trace_per_min = Scatter(x=df_PR['V_plc'],
                  y=df_PR['PAR(32)'],
                  name='Periodic Orbit',
                   mode='lines')
data = [trace_eq,trace_per]
layout = Layout(title="Bifurcation diagram",
               xaxis=dict(title='V_plc'),
               yaxis=dict(title='[Ca]'))
fig = Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
Is_1 = run(e='PKC_reduced_dat',c='PKC_reduced_model.per',dat='per5',ISW=1)
Is_2 = run(e='PKC_reduced_dat',c='PKC_reduced_model.per',dat='per5', DS='-',ISW=1,SP=['BP0'])
Is = Is_1 + Is_2

In [ ]:
df_Is = auto_to_csv(Is,'Is','IS')

In [ ]:
trace_eq = Scatter(x=df_eq['V_plc'],
                  y=df_eq['PAR(5)'],
                  name='Equilibria',
                  mode = 'lines')
trace_per = Scatter(x=df_PR['V_plc'],
                  y=df_PR['PAR(5)'],
                  name='Periodic Orbit',
                   mode='lines')
trace_per_min = Scatter(x=df_PR['V_plc'],
                  y=df_PR['PAR(32)'],
                  name='Periodic Orbit',
                   mode='lines')
trace_isola = Scatter(x=df_Is['V_plc'],
                  y=df_Is['PAR(5)'],
                  name='Isola',
                  mode='lines')
data = [trace_eq,trace_per,trace_per_min,trace_isola]
layout = Layout(title="Bifurcation diagram",
               xaxis=dict(title='V_plc'),
               yaxis=dict(title='[Ca]'))
fig = Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
df_eqs = []
df_PRs = []
initial = run(e='hepatocyte_npm',c='hepatocyte_npm.eq',DS='-',RL0 = 0.01, ICP =[2])
for R_act in np.linspace(0.04,0.5,200):
    first_eq = run(initial('EP2'),DS='-',RL0 = -10,RL1 = R_act, ICP =[1])
    Eq = run(first_eq('EP1'),c='hepatocyte_npm.eq',ICP =  [2,1,3,5,6,7,8,9,10,80])
    PO = []
    hb_count = 0
    for i in range(len(Eq())):
        if Eq()[i]['TY'] == 'HB':
            hb_count += 1
            PO = PO + run(Eq('HB'+ str(hb_count)),c ='hepatocyte_npm.per',ICP =  [2,11,1,3,5,6,7,8,9,10,80],ILP = 0,ISP = 0)
#     if Eq('HB2'):
#         PO1 = run(Eq('HB1'),c ='hepatocyte_npm.per',ICP =  [2,11,1,3,5,6,7,8,9,10,80],ILP = 0,ISP = 0)
#         PO1 = PO1 + run(Eq('HB2'),c ='hepatocyte_npm.per',ICP =  [2,11,1,3,5,6,7,8,9,10,80],ILP = 0,ISP = 0)
#     elif Eq('HB1'):
#         PO1 = run(Eq('HB1'),c ='hepatocyte_npm.per',ICP =  [2,11,1,3,5,6,7,8,9,10,80],ILP = 0,ISP = 0)
    df_eqs.append(auto_to_csv(Eq,'Eq_R_act'+str(R_act),'EQ'))
    df_PRs.append(auto_to_csv(PO,'PO_R_act'+str(R_act),'PR'))
    

In [ ]:
len(Eq())


In [ ]:
trace_eq = Scatter(x=df_eqs[8]['ct'],
                  y=df_eqs[8]['PAR(5)'],
                  name='Equilibria',
                  mode = 'lines')
trace_per = Scatter(x=df_PRs[8]['ct'],
                  y=df_PRs[8]['PAR(5)'],
                  name='Periodic Orbit',
                   mode='lines')
trace_per_min = Scatter(x=df_PRs[8]['ct'],
                  y=df_PRs[8]['PAR(8)'],
                  name='Periodic Orbit',
                   mode='lines')
data = [trace_eq,trace_per,trace_per_min]
layout = Layout(title="Bifurcation diagram",
               xaxis=dict(title='Agonist'),
               yaxis=dict(title='[Ca]'))
fig = Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
df_per1[1:-1].to_csv('per1.csv')

In [ ]:
df_per2[1:-1].to_csv('per2.csv')

In [ ]:
df_per2[1:-1]